In [1]:
# from __future__ import absolute_import, division, print_function
import pandas as pd
# load the data into panda dataframe
data_file_name = "~/Dropbox/DataRepo/AmazonReviews_Musical_Instruments/reviews_Musical_Instruments.json"
raw_df = pd.read_json(data_file_name, lines=True)
print("Data loaded")

Data loaded


In [2]:
# View information about the data
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500176 entries, 0 to 500175
Data columns (total 9 columns):
reviewerID        500176 non-null object
asin              500176 non-null object
reviewerName      497590 non-null object
helpful           500176 non-null object
reviewText        500176 non-null object
overall           500176 non-null int64
summary           500176 non-null object
unixReviewTime    500176 non-null int64
reviewTime        500176 non-null object
dtypes: int64(2), object(7)
memory usage: 34.3+ MB


In [3]:
# Convert all the review text into a long string and print its length
raw_corpus = u''.join(raw_df['reviewText']+" ")
print("Raw Corpus contains {0:,} characters".format(len(raw_corpus)))

Raw Corpus contains 241,653,315 characters


In [6]:
# import natural language toolkit
import nltk
# download the punkt tokenizer
nltk.download('punkt')
print("The punkt tokenizer is downloaded")

The punkt tokenizer is downloaded


[nltk_data] Downloading package punkt to /home/vineeth/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
# Load the punkt tokenizer
tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")
print("The punkt tokenizer is loaded")
# we tokenize the raw string into raw sentences
raw_sentences = tokenizer.tokenize(raw_corpus)
print("We have {0:,} raw sentences".format(len(raw_sentences)))

The punkt tokenizer is loaded
We have 2,492,388 raw sentences


In [8]:
import re, tqdm
# Clean and split sentence into words
def clean_and_split_str(string):
    strip_special_chars = re.compile("[^A-Za-z]+")
    string = re.sub(strip_special_chars, " ", string)
    return string.strip().split()

In [7]:
# clean each raw sentences and build the list of sentences
sentences = []
for raw_sent in raw_sentences:
    if len(raw_sent) > 0:
        sentences.append(clean_and_split_str(raw_sent))
print("We have {0:,} clean sentences".format(len(sentences)))

We have 2,492,388 clean sentences


In [8]:
sentences[10]

[u'Aida',
 u'was',
 u'first',
 u'performed',
 u'at',
 u'the',
 u'Khedivial',
 u'Opera',
 u'House',
 u'in',
 u'Cairo',
 u'on',
 u'December',
 u'conducted',
 u'by',
 u'Giovanni',
 u'Bottesini',
 u'Overview',
 u'Aida',
 u'an',
 u'Ethiopian',
 u'princess',
 u'is',
 u'captured',
 u'and',
 u'brought',
 u'into',
 u'slavery',
 u'in',
 u'Egypt']

In [9]:
print(raw_sentences[20])
print()
print(sentences[20])

.the music slowed at times for structural rigour.. .

[u'the', u'music', u'slowed', u'at', u'times', u'for', u'structural', u'rigour']


In [10]:
token_count = sum([len(sentence) for sentence in sentences])
print("The dataset corpus contains {0:,} tokens".format(token_count))

The dataset corpus contains 44,962,028 tokens


In [11]:
import multiprocessing

#Dimensionality of the resulting word vectors
num_features = 300

#Minimum word count threshold
min_word_count = 800

#Number of threads to run in parallel
num_workers = multiprocessing.cpu_count()

#Context window length
context_size = 4

#Seed for the RNG, to make the result reproducible
seed = 1

In [12]:
import gensim

doc2vec_model = gensim.models.doc2vec.Doc2Vec(
    seed=seed,
    workers=num_workers, 
    size=num_features, 
    min_count=min_word_count, 
    window=context_size)

/home/vineeth/anaconda2/lib/python2.7/site-packages/gensim/models/doc2vec.py:359: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [13]:
sentences = [gensim.models.doc2vec.TaggedDocument(s,[i]) for i,s in enumerate(sentences)]

In [14]:
sentences[0]

TaggedDocument(words=[u'The', u'portfolio', u'is', u'fine', u'except', u'for', u'the', u'fact', u'that', u'the', u'last', u'movement', u'of', u'sonata', u'is', u'missing'], tags=[0])

In [15]:
doc2vec_model.build_vocab(sentences)
print("The vocabword2vec_model.iterulary is built")
print("Word2Vec vocabulary length: ", len(doc2vec_model.wv.vocab))

The vocabword2vec_model.iterulary is built


NameError: name 'word2vec_model' is not defined

In [22]:
#Start training the model
doc2vec_model.train(documents=sentences,total_examples=doc2vec_model.corpus_count,epochs=doc2vec_model.iter)
print("Training finished")

/home/vineeth/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


Training finished


In [23]:
#Save the model
doc2vec_model.save("reviews_Musical_Instruments.d2v")
print("Model saved")

Model saved


In [ ]:
doc

In [11]:
import gensim

word2vec_model = gensim.models.word2vec.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers, 
    size=num_features, 
    min_count=min_word_count, 
    window=context_size)

In [12]:
word2vec_model.build_vocab(sentences=sentences)
print("The vocabword2vec_model.iterulary is built")
print("Word2Vec vocabulary length: ", len(word2vec_model.wv.vocab))

The vocabword2vec_model.iterulary is built
Word2Vec vocabulary length:  96014


In [13]:
#Start training the model
word2vec_model.train(sentences=sentences,epochs=word2vec_model.iter)
print("Training finished")

/home/vineeth/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


TypeError: train() got an unexpected keyword argument 'size'

In [14]:
#Start training the model
word2vec_model.train(sentences=sentences,total_examples=word2vec_model.corpus_count,epochs=word2vec_model.iter)
print("Training finished")

/home/vineeth/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


Training finished


In [33]:
#Save the model
doc2vec_model.save("reviews_Musical_Instruments.d2v")
print("Model saved")

Model saved
